In [43]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import xgboost
import numpy as np
from sklearn.metrics import explained_variance_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

pd.set_option('display.max_columns', 500)

In [44]:
df_full = pd.read_csv('./yvw_first_break_months.csv', sep='\t', low_memory=False)
df_full.head(1)

,Asset ID,count,Event Time,Event Date,Class Structure,Class Structure ID,Material,Nominal Size (mm),Soil Class,Soil Description,Grant Description,Pipe Inside Diameter (mm),Pipe Material,Pipe Shape,Construction Date (YYYYMMDD),Pipe Lining,Date Insulated,Tapping Status,Road Name,Road Type,Pipe Length (m),Pipe Status,Height Start of Pipe (meters above sea level),Height End of Pipe (meters above sea level),Latitude Start of Pipe,Lattitude End of Pipe,Longitude Start of Pipe,Longitude End of Pipe,Distribution Zone ID,Water Quality Zone ID,Description,Failure Code,Failure Description,Job Plan,Job Plan Number,Service Location Id,Formatted Address,City,Post Code,Latitude,Longitude,months_break
0,100259,1,15:54:00.0000000,2004-04-05,Repair Burst Water Main,WRBRST,MEDIUM DENSITY POLYETHYLENE,125,EX,Moderate shrink/swell potential.,Clay.,125,MEDIUM DENSITY POLYETHYLENE,CIRCULAR,2003-07-10,NONE,(null),YES,MOUNT DANDENONG TOURIST,ROAD,396.8047,INTEGRITY,500.0,484.19,-37.844721,-37.841505,145.360402,145.358637,1.550536e+09,1.125029e+09,REP LK 1.5M DEEP NON MS 100DIA,UNKN,UNKN,NaN,NaN,1399527.0,1417 MT DANDENONG TOURIST ROAD OLINDA 3788,OLINDA,3788.0,-37.84193818,145.35890900,3


In [45]:
df = df_full[['Class Structure ID', 'Material', 'Nominal Size (mm)', 'Soil Class', 'Grant Description', 'Pipe Inside Diameter (mm)', 'Pipe Material', 'Pipe Lining', 'Pipe Length (m)', 'Height Start of Pipe (meters above sea level)', 'Height End of Pipe (meters above sea level)', 'months_break']]
df.head()
# df[['months_break']].median()

,Class Structure ID,Material,Nominal Size (mm),Soil Class,Grant Description,Pipe Inside Diameter (mm),Pipe Material,Pipe Lining,Pipe Length (m),Height Start of Pipe (meters above sea level),Height End of Pipe (meters above sea level),months_break
0,WRBRST,MEDIUM DENSITY POLYETHYLENE,125,EX,Clay.,125,MEDIUM DENSITY POLYETHYLENE,NONE,396.8047,500.0000,484.19,3
1,WRBRST,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",125,EX,Clay.,125,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",NONE,291.2699,50.0000,34.94,7
2,WRBRST,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),300,VE,Expansive Clay.,300,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),NONE,147.2552,151.0000,155.00,21
3,WRBRST,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),100,VE,Expansive Clay.,100,UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),NONE,62.3840,147.6768,147.50,3
4,WRBRST,MEDIUM DENSITY POLYETHYLENE,63,VE,Expansive Clay.,63,MEDIUM DENSITY POLYETHYLENE,NONE,56.0505,114.7660,111.10,17


## Remove uneeded values

In [46]:
df = df[df['Pipe Material'] != 'ASBESTOS CEMENT(INCL HARDYFLEX)']
df = df[df['Pipe Material'] != 'DUCTILE IRON CEMENT LINED']
df = df[df['Pipe Material'] != 'GLASS REINFORCED PLASTIC(INCL HOBAS)']
df = df[df['Pipe Material'] != 'DUCTILE IRON(SPHERIOD GRAPHITE CAST IRON)']
df = df[df['Pipe Material'] != 'CAST IRON CEMENT LINED']
df = df[df['Pipe Material'] != 'COPPER']
# df.groupby('Pipe Material')['Asset ID'].nunique()


In [47]:
df = df[df['Soil Class'] != '(null)']
# df.groupby('Soil Class')['Asset ID'].nunique()

In [48]:
# df.groupby('Grant Description')['Asset ID'].nunique()

## Split the training and the test set

In [49]:
train_pre, test_pre = train_test_split(df, test_size = 0.3)

## Turn the data into dataframes and perform the iterative data splitting

In [50]:
test = pd.DataFrame(columns=test_pre.columns)
train = pd.DataFrame(columns=train_pre.columns)

In [51]:
for index, row in test_pre.iterrows():
    end_month = int(row['months_break'])
    for month in range(1, end_month + 1):
        if month == end_month:
            new_row = row
            new_row['age'] = month
            new_row['will_break'] = True
            test = test.append(new_row)

In [52]:
for index, row in train_pre.iterrows():
    end_month = int(row['months_break'])
    for month in range(1, end_month + 1):
        new_row = row
        new_row['age'] = month
        if month >= end_month - 5:
            new_row['will_break'] = True
        else:
            new_row['will_break'] = False

        train = train.append(new_row)

In [53]:
test

,Class Structure ID,Material,Nominal Size (mm),Soil Class,Grant Description,Pipe Inside Diameter (mm),Pipe Material,Pipe Lining,Pipe Length (m),Height Start of Pipe (meters above sea level),Height End of Pipe (meters above sea level),months_break,age,will_break
1651,WRBRST,UPVC - MODIFIED,300,EX,Clay.,300,UPVC - MODIFIED,NONE,376.0259,71.0600,89.0000,8,8.0,1.0
78,WRBRST,UPVC - MODIFIED,225,EX,Clay.,225,UPVC - MODIFIED,NONE,71.8996,133.0900,132.3700,30,30.0,1.0
1325,WRBRST,UPVC - MODIFIED,150,EX,Clay.,150,UPVC - MODIFIED,NaN,51.8583,101.9300,98.3600,4,4.0,1.0
734,WRBRST,UPVC - MODIFIED,225,EX,Clay.,225,UPVC - MODIFIED,NONE,26.0560,117.4820,117.4250,8,8.0,1.0
1335,WRBRST,UPVC - MODIFIED,150,EX,Clay.,150,UPVC - MODIFIED,NONE,133.5788,150.1910,135.6890,8,8.0,1.0
1513,WRBRST,MEDIUM DENSITY POLYETHYLENE,125,EX,Clay.,125,MEDIUM DENSITY POLYETHYLENE,NONE,154.1927,89.6398,87.8100,32,32.0,1.0
719,WRBRST,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",63,EX,Clay.,63,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",NONE,13.0544,128.9600,130.2700,10,10.0,1.0
2038,WRBRST,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",63,EX,Clay.,63,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",NONE,73.8046,126.8500,128.0900,3,3.0,1.0
2089,WRBRST,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",125,EX,Clay.,125,"POLYETHYLENE(SOLID WALL, HIGH DENSITY)",NONE,81.3250,54.2300,57.4700,29,29.0,1.0
1789,WRBRST,UPVC - MODIFIED,150,VE,Expansive Clay.,150,UPVC - MODIFIED,NONE,148.1662,88.4720,94.2470,31,31.0,1.0


## Get the train data in the correct formats

In [54]:
# test['will_break'] = test['will_break'].astype('str')
test['months_break'] = test['months_break'].astype('int')
# test['will_break'] = test['will_break'].astype('float').astype('int').astype('bool')
test['Nominal Size (mm)'] = test['Nominal Size (mm)'].astype('float')
test['Pipe Inside Diameter (mm)'] = test['Pipe Inside Diameter (mm)'].astype('float')



# train['will_break'] = train['will_break'].astype('str')
train['months_break'] = train['months_break'].astype('int')
# train['will_break'] = train['will_break'].astype('float').astype('int').astype('bool')
train['Nominal Size (mm)'] = train['Nominal Size (mm)'].astype('float')
train['Pipe Inside Diameter (mm)'] = train['Pipe Inside Diameter (mm)'].astype('float')


train.dtypes
# test_ready = pd.get_dummies(test)
# test_ready.

Class Structure ID                                object
Material                                          object
Nominal Size (mm)                                float64
Soil Class                                        object
Grant Description                                 object
Pipe Inside Diameter (mm)                        float64
Pipe Material                                     object
Pipe Lining                                       object
Pipe Length (m)                                  float64
Height Start of Pipe (meters above sea level)    float64
Height End of Pipe (meters above sea level)      float64
months_break                                       int64
age                                              float64
will_break                                       float64
dtype: object

In [55]:
test_ready = pd.get_dummies(test)
train_ready = pd.get_dummies(train)

# test_ready.sort_values('will_break')

In [56]:
test_ready = test_ready.drop('months_break', 1)
train_ready = train_ready.drop('months_break', 1)
train_ready

,Nominal Size (mm),Pipe Inside Diameter (mm),Pipe Length (m),Height Start of Pipe (meters above sea level),Height End of Pipe (meters above sea level),age,will_break,Class Structure ID_WRBRST,Class Structure ID_WRLEAK,Class Structure ID_WRPWSL,Material_MEDIUM DENSITY POLYETHYLENE,Material_MILD STEEL CEMENT LINED,Material_OPVC - MOLECULAR ORIENTED,Material_PLASTIC,"Material_POLYETHYLENE(SOLID WALL, HIGH DENSITY)",Material_UPVC - MODIFIED,Material_UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),Soil Class_EX,Soil Class_SE,Soil Class_ST,Soil Class_VE,Grant Description_Clay.,Grant Description_Expansive Clay.,Grant Description_Sand.,Grant Description_Silty Soil.,Pipe Material_MEDIUM DENSITY POLYETHYLENE,Pipe Material_MILD STEEL CEMENT LINED,Pipe Material_OPVC - MOLECULAR ORIENTED,Pipe Material_PLASTIC,"Pipe Material_POLYETHYLENE(SOLID WALL, HIGH DENSITY)",Pipe Material_UPVC - MODIFIED,Pipe Material_UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),Pipe Lining_NONE
314,100.0,100.0,78.3920,150.2330,154.943,1.0,0.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
314,100.0,100.0,78.3920,150.2330,154.943,2.0,0.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
314,100.0,100.0,78.3920,150.2330,154.943,3.0,0.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
314,100.0,100.0,78.3920,150.2330,154.943,4.0,0.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
314,100.0,100.0,78.3920,150.2330,154.943,5.0,0.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
314,100.0,100.0,78.3920,150.2330,154.943,6.0,0.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
314,100.0,100.0,78.3920,150.2330,154.943,7.0,0.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
314,100.0,100.0,78.3920,150.2330,154.943,8.0,0.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
314,100.0,100.0,78.3920,150.2330,154.943,9.0,0.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1
314,100.0,100.0,78.3920,150.2330,154.943,10.0,0.0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1


In [57]:
y = train_ready.pop('will_break')
y_test = test_ready.pop('will_break')

## Testing Feature Important (not needed for final model)

In [58]:
# gbt = GradientBoostingClassifier(max_depth=8, n_estimators=1000, subsample=.5, random_state=0, learning_rate=.001)
# gbt.fit(train_ready, y)

In [59]:
# cross_val_score(gbt, test_ready, y_test, scoring='accuracy').mean()

In [60]:
# test_ready

In [61]:
# feature_importance = gbt.feature_importances_
# feature_importance = 100.0 * (feature_importance / feature_importance.max())
# features_df = pd.DataFrame({"Features": train_ready.columns, "Importance Score": feature_importance})
# features_df.sort_values("Importance Score", inplace = True, ascending = False)

# features_df.head()
# print(features_df)

In [62]:
# cross_val_score(gbt, sanity_pre_train_ready, sanity_y, scoring='accuracy').mean()

## Set the xgboost model parameters and train the model

In [63]:
xg = xgboost.XGBClassifier(min_child_weight=2, subsample=.2, n_estimators=15000, max_depth=10,colsample_bytree=.2)
xg.fit(train_ready, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.2,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=10,
       min_child_weight=2, missing=None, n_estimators=15000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.2)

In [64]:
test_ready

,Nominal Size (mm),Pipe Inside Diameter (mm),Pipe Length (m),Height Start of Pipe (meters above sea level),Height End of Pipe (meters above sea level),age,Class Structure ID_WRBRST,Class Structure ID_WRLEAK,Class Structure ID_WRPWSL,Material_MEDIUM DENSITY POLYETHYLENE,Material_MILD STEEL CEMENT LINED,Material_OPVC - MOLECULAR ORIENTED,Material_PLASTIC,"Material_POLYETHYLENE(SOLID WALL, HIGH DENSITY)",Material_UPVC - MODIFIED,Material_UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),Soil Class_EX,Soil Class_SE,Soil Class_ST,Soil Class_VE,Grant Description_Clay.,Grant Description_Expansive Clay.,Grant Description_Sand.,Grant Description_Silty Soil.,Pipe Material_MEDIUM DENSITY POLYETHYLENE,Pipe Material_MILD STEEL CEMENT LINED,Pipe Material_OPVC - MOLECULAR ORIENTED,Pipe Material_PLASTIC,"Pipe Material_POLYETHYLENE(SOLID WALL, HIGH DENSITY)",Pipe Material_UPVC - MODIFIED,Pipe Material_UPVC-THICK WALLED PRESSURE PIPE(INCL BLUE BRUTE),Pipe Lining_NONE
1651,300.0,300.0,376.0259,71.0600,89.0000,8.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
78,225.0,225.0,71.8996,133.0900,132.3700,30.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
1325,150.0,150.0,51.8583,101.9300,98.3600,4.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
734,225.0,225.0,26.0560,117.4820,117.4250,8.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
1335,150.0,150.0,133.5788,150.1910,135.6890,8.0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1
1513,125.0,125.0,154.1927,89.6398,87.8100,32.0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1
719,63.0,63.0,13.0544,128.9600,130.2700,10.0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
2038,63.0,63.0,73.8046,126.8500,128.0900,3.0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
2089,125.0,125.0,81.3250,54.2300,57.4700,29.0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1
1789,150.0,150.0,148.1662,88.4720,94.2470,31.0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1


In [65]:
y_pred = xg.predict(test_ready)
predictions = [round(value) for value in y_pred]
accuracy = accuracy_score(y_test, predictions)
#print("Accuracy: %.2f%%" % (accuracy * 100.0))

## Look at the features of the model

In [66]:
feature_importance = xg.feature_importances_
feature_importance = 100.0 * (feature_importance / feature_importance.max())
features_df = pd.DataFrame({"Features": train_ready.columns, "Importance Score": feature_importance})
features_df.sort_values("Importance Score", inplace = True, ascending = False)

features_df.head()
print(features_df)


                                             Features  Importance Score
2                                     Pipe Length (m)        100.000000
3       Height Start of Pipe (meters above sea level)         99.292183
4         Height End of Pipe (meters above sea level)         98.541298
5                                                 age         77.154182
0                                   Nominal Size (mm)         19.357849
1                           Pipe Inside Diameter (mm)         16.509523
15  Material_UPVC-THICK WALLED PRESSURE PIPE(INCL ...          7.306965
16                                      Soil Class_EX          6.445738
30  Pipe Material_UPVC-THICK WALLED PRESSURE PIPE(...          6.412545
20                            Grant Description_Clay.          5.496595
14                           Material_UPVC - MODIFIED          5.494801
19                                      Soil Class_VE          5.110838
29                      Pipe Material_UPVC - MODIFIED          4

In [70]:
test_ready.to_csv('test_frame.csv', sep='\t', encoding='utf-8')

In [68]:
np.savetxt("predictions.csv", y_pred, delimiter=",")